<a href="https://colab.research.google.com/github/HalgasAdrian/CS5230-Coursework/blob/main/HW5B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>